In [ ]:
from google.colab import userdata

my_secret_key = userdata.get("comet_ml_api_key")

In [ ]:
! pip install --upgrade comet_ml --quiet
from comet_ml import start

experiment = start(
    api_key=my_secret_key, project_name="wav2vec-mer", workspace="nikzagl"
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.8/725.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 34.0 MB/s eta 0:00:00


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/nikzagl/wav2vec-mer/b5382a2d94e347228601a0d94cc01bc5

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Mounted at /content/drive/


In [ ]:
from google.colab import drive

drive.mount("/content/drive/")
%cd "/content/drive/MyDrive/MER Project"

/content/drive/MyDrive/MER Project


In [ ]:
import fnmatch
import os

files = []
for root, dirnames, filenames in os.walk("MP3-Example"):
    for filename in fnmatch.filter(filenames, "*mp3"):
        files.append(os.path.join(root, filename))

In [ ]:
import re

track_ids = list()
for file in files:
    track_id = re.split(r"\.|-", file)[-2]
    track_ids.append(track_id)

In [ ]:
import pandas as pd

musicinfo_df = pd.read_csv("Music Info.csv")

In [ ]:
track_ids = pd.DataFrame(track_ids).rename({0: "track_id"}, axis=1)
track_ids = track_ids.join(musicinfo_df.set_index("track_id"), on="track_id")

In [ ]:
def classify_emotions(energy, valence):
    e = energy - 0.5
    v = valence - 0.5
    if v > 0 and e > 0:
        return "Joy"
    elif v < 0 and e > 0:
        return "Anger"
    elif v < 0 and e < 0:
        return "Sad"
    else:
        return "Pleasure"

In [ ]:
emotions = track_ids.apply(lambda x: classify_emotions(x.energy, x.valence), axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
labels = le.fit_transform(emotions)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
tensor_df = torch.load("waveforms_tensor-1600.pt")

In [ ]:
from sklearn.model_selection import train_test_split

tensor_df_train, tensor_df_test, labels_train, labels_test = train_test_split(
    tensor_df.T, labels, random_state=42
)

In [ ]:
train_dataset = TensorDataset(tensor_df_train, torch.tensor(labels_train))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base-960h", num_labels=4
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import random

hyper_params = {"seed": 42, "batch_size": 16, "num_epochs": 25, "learning_rate": 1e-5}


# Logging hyperparamters
experiment.log_parameters(hyper_params)
random.seed(hyper_params["seed"])
torch.manual_seed(hyper_params["seed"])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
import numpy as np

# Получение весов классов
class_weights = compute_class_weight(
    "balanced", classes=np.unique(labels_train), y=labels_train
)

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size=hyper_params["batch_size"], shuffle=True
)
test_dataset = TensorDataset(tensor_df_test, torch.tensor(labels_test))
test_loader = DataLoader(
    test_dataset, shuffle=False, batch_size=hyper_params["batch_size"]
)
# Оптимизатор
optimizer = torch.optim.Adam(model.parameters(), hyper_params["learning_rate"])
loss_function = torch.nn.CrossEntropyLoss(
    weight=torch.tensor(class_weights).float().to(device)
)
num_epochs = 25
model.train()
# Цикл обучения
step = 0
with experiment.train():
    for epoch in range(num_epochs):
        losses = list()
        model.train()
        for audio, labels in train_loader:
            optimizer.zero_grad()
            audio = audio.to(device)
            labels = labels.to(device)
            outputs = model(audio)
            loss = loss_function(outputs.logits, labels)
            losses.append(loss.item())
            experiment.log_metric(name="loss", value=loss.item(), step=step)
            loss.backward()
            optimizer.step()
            step += 1
            del audio
            del labels
        model.eval()
        accuracy_score = 0
        labels_pred = list()
        for audio, labels in test_loader:
            audio = audio.to(device)
            _, outputs = torch.max(model(audio).logits, 1)
            labels_pred += outputs.cpu().tolist()
        f1_macro = f1_score(labels_test, labels_pred, average="macro")
        experiment.log_metric(name="f1_score_macro", value=f1_macro, epoch=epoch + 1)

In [ ]:
model.eval()

Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
labels_pred = list()
for audio, labels in test_loader:
    audio = audio.to(device)
    _, outputs = torch.max(model(audio).logits, 1)
    labels_pred += outputs.cpu().tolist()
f1_weighted = f1_score(labels_test, labels_pred, average="weighted")
print(f1_weighted)

0.631876623723782


In [ ]:
from sklearn.metrics import balanced_accuracy_score

In [ ]:
print(classification_report(labels_test, labels_pred))
experiment.log_confusion_matrix(labels_test, labels_pred, labels=le.classes_)

              precision    recall  f1-score   support

           0       0.65      0.71      0.68       131
           1       0.66      0.61      0.64       129
           2       0.36      0.60      0.45        35
           3       0.79      0.53      0.63        80

    accuracy                           0.63       375
   macro avg       0.61      0.61      0.60       375
weighted avg       0.66      0.63      0.63       375



{'web': 'https://www.comet.com/api/asset/download?assetId=aeb23c6289fb4572bc5a36fea7cee61c&experimentKey=b5382a2d94e347228601a0d94cc01bc5',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=aeb23c6289fb4572bc5a36fea7cee61c&experimentKey=b5382a2d94e347228601a0d94cc01bc5',
 'assetId': 'aeb23c6289fb4572bc5a36fea7cee61c'}

In [ ]:
labels_test_decoded = le.inverse_transform(labels_test)
labels_pred_decoded = le.inverse_transform(labels_pred)

In [ ]:
print(classification_report(labels_test_decoded, labels_pred_decoded))

              precision    recall  f1-score   support

       Anger       0.65      0.71      0.68       131
         Joy       0.66      0.61      0.64       129
    Pleasure       0.36      0.60      0.45        35
         Sad       0.79      0.53      0.63        80

    accuracy                           0.63       375
   macro avg       0.61      0.61      0.60       375
weighted avg       0.66      0.63      0.63       375



In [ ]:
print(balanced_accuracy_score(labels_test, labels_pred))

0.6118316912243328


In [ ]:
print(f1_score(labels_test, labels_pred, average="macro"))

0.5979619671409759


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/MER Project/wav2vec_weights.pth")

In [ ]:
experiment.end()